In [3]:
from bs4 import BeautifulSoup
import requests
import datetime as dt
from PIL import Image
import helpers as hf
import yaml
from unidecode import unidecode
import pandas as pd
from PIL import Image, ImageOps, ImageDraw, ImageFont
import json


with open("design_config.yml", "r") as f:
    design_config = yaml.safe_load(f)

design_config,shop_config = hf.get_config('cbb')
cfbd_loader = hf.cfbp_handler(design_config['cfbd_api'])

In [32]:
big_body = """
{
  product(id:"gid://shopify/Product/prod_id") {
    title
    media(first:5) {
      edges {
        node {
          ... fieldsForMediaTypes
        }
      }
    }
  }
}

fragment fieldsForMediaTypes on Media {
  alt
  mediaContentType
  preview {
    image {
      id
      altText
      url
    }
  }
  status
  ... on Video {
    id
    sources {
      format
      height
      mimeType
      url
      width
    }
    originalSource {
      format
      height
      mimeType
      url
      width
    }
  }
  ... on ExternalVideo {
    id
    host
    embeddedUrl
  }
  ... on Model3d {
    sources {
      format
      mimeType
      url
    }
    originalSource {
      format
      mimeType
      url
    }
  }
  ... on MediaImage {
    id
    image {
      altText
      url
    }
  }
}
"""

big_body2 = """
{
  product(id:"gid://shopify/Product/prod_id") {
    title
    media(first:10) {
      edges {
        node {
        }
      }
    }
  }
}
"""
mid_body="""{
"query": "mutation productReorderMedia($id: ID!, $moves: [MoveInput!]!) { productReorderMedia(id: $id, moves: $moves) { job { id } } }",
 "variables": {
    "id": "gid://shopify/Product/prod_id",
    "moves": [
      {
        "id": "image0",
        "newPosition": "2"
      },
      {
        "id": "image2",
        "newPosition": "0"
      }
    ]
  }
}"""


In [4]:
headers_shopify = {
            "X-Shopify-Access-Token": shop_config["shopify_access"],
            "Content-Type": "application/json",
        }

In [29]:
def recursive_get_prods(since_id=0,all_products=[]):
    url = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2023-01/products.json?since_id={since_id}"
    response = requests.get(url=url,headers = headers_shopify)
    products = json.loads(response.text)['products']
    print(response.status_code)
    if len(products)==0:
        return all_products
    else:
        # new products were found, there could be more in the next iteration
        all_products.extend(products)
        return recursive_get_prods(since_id=products[-1]["id"],all_products=all_products)

In [30]:
all_prods = recursive_get_prods(since_id=0,all_products=[])

200
200


In [33]:
for prod in all_prods:
    prod_id = prod['id']
    prod_gql = prod['admin_graphql_api_id']
    body = big_body.replace('prod_id',str(prod_id))
    # now that I have the images I can reorder them
    url = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2024-01/graphql.json"
    # from this response i can get my images
    response = requests.post(url=url,headers = headers_shopify, json = {"query":body})

    images = json.loads(response.text)['data']['product']['media']['edges']
    image0 = images[0]['node']['id']
    image2 = images[2]['node']['id']

    # image0 = prod['images'][0]['admin_graphql_api_id']
    # image2 = prod['images'][2]['admin_graphql_api_id']

    query = """
    mutation productReorderMedia($id: ID!,
    $moves: [MoveInput!]!) {
      productReorderMedia(id: $id, moves:
      $moves) {
        job {
        id
        }
      }
    }"""


    # plug my images and id into this and it should update
    variables = {
        "id": f"{prod_gql}",
        "moves": [
            {
            "id": f"{image0}",
            "newPosition": "2"
            },
            {
            "id": f"{image2}",
            "newPosition": "0"
            }
        ]
        }
    response2 = requests.post(url=url, json={'query': query , 'variables': variables}, headers= headers_shopify)



    if response2.status_code == 200:
        print(response2.text)

{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/12072c7d-6726-42d8-aee8-1e437d7afba8"}}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":2000.0,"currentlyAvailable":1990,"restoreRate":100.0}}}}
{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/db922f82-fa2e-4df9-bf75-a902ab03e02c"}}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":2000.0,"currentlyAvailable":1990,"restoreRate":100.0}}}}
{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/b8169912-e01d-46e5-9fc8-2278201cabad"}}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":2000.0,"currentlyAvailable":1990,"restoreRate":100.0}}}}
{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/053c3dee-fe07-4488-b916-b87f617824b6"}}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailabl

In [57]:
for prod in all_prods:
  prod_id = prod['id']
  prod_gql = prod['admin_graphql_api_id']
  body = big_body.replace('prod_id',str(prod_id))
  # now that I have the images I can reorder them
  url = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2024-01/graphql.json"
  # from this response i can get my images
  response = requests.post(url=url,headers = headers_shopify, json = {"query":body})

  images = json.loads(response.text)['data']['product']['media']['edges']
  image0 = images[0]['node']['id']
  image2 = images[2]['node']['id']

  q="""
  mutation reorderProductMedia {
    productReorderMedia(
      id: "gidp",
      moves: [
      {
        id: "gid0",
        newPosition: "2"
      },
      {
        id: "gid2",
        newPosition: "0"
      }
    ]) {
      job {
        id
        done
      }
      mediaUserErrors {
        code
        field
        message
      }
    }
  }
  """.replace('gidp',prod_gql).replace('gid0',image0).replace('gid2',image0)


  graphql_url =f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2023-04/graphql.json"

  response = requests.post(graphql_url, headers=headers_shopify, json={'query': q})
  print(response.text)



{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/96b7ebc4-d1cd-4fa9-a1dd-992e67c6e9cf","done":false},"mediaUserErrors":[]}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":2000.0,"currentlyAvailable":1990,"restoreRate":100.0}}}}
{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/a6cd6383-f63a-4e51-af6e-f66afd514664","done":false},"mediaUserErrors":[]}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":2000.0,"currentlyAvailable":1990,"restoreRate":100.0}}}}
{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/8f15fc83-856d-4a4e-b0fd-039da32d091f","done":false},"mediaUserErrors":[]}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":2000.0,"currentlyAvailable":1990,"restoreRate":100.0}}}}
{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/1feda553-0662-405c-a3c1-bdadcd4a7f8e","d

In [54]:
check_q='''{
"query { job(id: "gid://shopify/Job/87147bdd-eda8-4a1b-bd2c-63507e1ef074") { id done } }"
}'''
check_url = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2024-01/graphql.json"
check_response = requests.post(graphql_url, headers=headers_shopify, json={'query': check_q})

In [55]:
response.text

'{"data":{"productReorderMedia":{"job":{"id":"gid://shopify/Job/78298459-6ec4-421a-a0c8-42c1fe5572e8","done":false},"mediaUserErrors":[]}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":2000.0,"currentlyAvailable":1990,"restoreRate":100.0}}}}'